## Part 1: Preprocessing

In [64]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [65]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [53]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [54]:
# Create a list of at least 10 column names to use as X data
columns = [
    'Education',
    'Age',
    'DistanceFromHome',
    'JobSatisfaction',
    'OverTime',
    'StockOptionLevel',
    'WorkLifeBalance',
    'YearsAtCompany',
    'YearsSinceLastPromotion',
    'NumCompaniesWorked'
]

# Create X_df using your selected columns
X_df = attrition_df[columns]

# Show the data types for X_df
X_df.dtypes


,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [55]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [56]:
X_train.head()

,Education,Age,DistanceFromHome,JobSatisfaction,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
464,3,37,1,4,No,0,4,6,1,4
512,4,27,3,4,No,0,3,4,1,0
480,4,30,12,1,No,1,4,1,0,1
680,4,36,7,4,No,0,3,6,0,2
1023,2,56,1,1,No,1,4,3,1,2


In [57]:
print(X_train['OverTime'].value_counts())

OverTime
No     798
Yes    304
Name: count, dtype: int64


In [58]:
# Convert your X data to numeric data types however you see fit
overtime_train = pd.get_dummies(X_train['OverTime'], prefix='OverTime')
overtime_test = pd.get_dummies(X_test['OverTime'], prefix='OverTime')

# Drop the original "OverTime" column and add encoded version
X_train = X_train.drop(columns=['OverTime']).join(overtime_train)
X_test = X_test.drop(columns=['OverTime']).join(overtime_test)

# Ensure X_test has the same columns as X_train
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Add new code cells as necessary

# Check one of the new columns
print(X_train['OverTime_Yes'].head())


464     False
512     False
480     False
680     False
1023    False
Name: OverTime_Yes, dtype: bool


In [59]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [80]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])


# Create two new variables by applying the encoder to the training and testing data
department_train_encoded = department_encoder.transform(y_train[['Department']])
department_test_encoded = department_encoder.transform(y_test[['Department']])

# Convert to DataFrame
department_columns = department_encoder.get_feature_names_out(['Department'])
department_train_df = pd.DataFrame(department_train_encoded, columns=department_columns, index=y_train.index)
department_test_df = pd.DataFrame(department_test_encoded, columns=department_columns, index=y_test.index)


In [81]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder to the training and testing data
attrition_train_encoded = attrition_encoder.transform(y_train[['Attrition']])
attrition_test_encoded = attrition_encoder.transform(y_test[['Attrition']])

attrition_columns = attrition_encoder.get_feature_names_out(['Attrition'])

# Convert to DataFrames
attrition_train_df = pd.DataFrame(attrition_train_encoded, columns=attrition_columns, index=y_train.index)
attrition_test_df = pd.DataFrame(attrition_test_encoded, columns=attrition_columns, index=y_test.index)



## Part 2: Create, Compile, and Train the Model

In [82]:
# Find the number of columns in the X training data.
input_dim = X_train_scaled.shape[1]
print(f"Number of input features: {input_dim}")

# Create the input layer
shared_input = Input(shape=(input_dim,), name="shared_input")

# Create at least two shared layers
shared_dense_1 = Dense(units=32, activation='relu', name='shared_dense_1')(shared_input)
shared_dense_2 = Dense(units=16, activation='relu', name='shared_dense_2')(shared_dense_1)


Number of input features: 11


In [83]:
# Create a branch for Department with a hidden layer and an output layer
department_input = Input(shape=(department_train_df.shape[1],), name="department_input")

# Create the hidden layer
department_hidden = Dense(16, activation='relu', name="department_hidden")(department_input)

# Create the output layer
department_output = Dense(3, activation='softmax', name="department_output")(department_hidden)

In [84]:
# Create a branch for Attrition with a hidden layer and an output layer
attrition_input = Input(shape=(attrition_train_df.shape[1],), name="attrition_input")

# Create the hidden layer
attrition_hidden = Dense(8, activation='relu', name="attrition_hidden")(attrition_input)

# Create the output layer
attrition_output = Dense(1, activation='sigmoid', name="attrition_output")(attrition_hidden)



In [88]:
# Create the model

# Merge the shared and branch paths
merged = Concatenate(name="merged_layer")([shared_dense_2, department_hidden, attrition_hidden])

# Optional final hidden layer before outputs
final_hidden = Dense(16, activation='relu', name="final_hidden")(merged)

# Final output layer for Attrition (binary classification)
final_attrition_output = Dense(1, activation='sigmoid', name="attrition_output")(final_hidden)

# Final output layer for Department (multi-class classification)
final_department_output = Dense(3, activation='softmax', name="department_output")(final_hidden)

# Create the model with all 3 inputs and both outputs
model = Model(
    inputs=[shared_input, department_input, attrition_input],
    outputs=[final_attrition_output, final_department_output]
)

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'attrition_output': 'binary_crossentropy',
        'department_output': 'categorical_crossentropy'
    },
    metrics={
        'attrition_output': 'accuracy',
        'department_output': 'accuracy'
    }
)

# Summarize the model
model.summary()


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ shared_input        │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_dense_1      │ (None, 32)        │        384 │ shared_input[0][… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_input    │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_input     │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_dense_2      │ (None, 16)        │        528 │ shared_dense_1[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 16)        │         64 │ department_input… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 8)         │         24 │ attrition_input[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ merged_layer        │ (None, 40)        │          0 │ shared_dense_2[0… │
│ (Concatenate)       │                   │            │ department_hidde… │
│                     │                   │            │ attrition_hidden… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_hidden        │ (None, 16)        │        656 │ merged_layer[0][… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 1)         │         17 │ final_hidden[0][… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         51 │ final_hidden[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,724 (6.73 KB)

 Trainable params: 1,724 (6.73 KB)

 Non-trainable params: 0 (0.00 B)

In [89]:
# Train the model
# Convert Attrition column to binary (if not already done)
y_train_binary = y_train['Attrition'].map({'Yes': 1, 'No': 0})

# Train the model with two outputs
history = model.fit(
    x=[X_train_scaled, department_train_df, attrition_train_df],
    y={
        'attrition_output': y_train_binary,
        'department_output': department_train_df
    },
    epochs=20,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - attrition_output_accuracy: 0.8698 - attrition_output_loss: 0.4764 - department_output_accuracy: 0.6212 - department_output_loss: 0.9137 - loss: 1.3900 - val_attrition_output_accuracy: 0.8281 - val_attrition_output_loss: 0.3949 - val_department_output_accuracy: 0.6335 - val_department_output_loss: 0.8032 - val_loss: 1.1971
Epoch 2/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8484 - attrition_output_loss: 0.3438 - department_output_accuracy: 0.6635 - department_output_loss: 0.6910 - loss: 1.0350 - val_attrition_output_accuracy: 0.8326 - val_attrition_output_loss: 0.3348 - val_department_output_accuracy: 0.7149 - val_department_output_loss: 0.6208 - val_loss: 0.9551
Epoch 3/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_accuracy: 0.8697 - attrition_output_loss: 0.2589 - department_output_accuracy: 0.7975 - department_output_loss: 0.5334 - loss: 0.7925 - val_attrition_output_accuracy: 0.8778 - va

In [90]:
# Evaluate the model with the testing data
# Convert Attrition column in y_test to binary
y_test_binary = y_test['Attrition'].map({'Yes': 1, 'No': 0})

# Evaluate the model with both outputs
results = model.evaluate(
    x=[X_test_scaled, department_test_df, attrition_test_df],
    y={
        'attrition_output': y_test_binary,
        'department_output': department_test_df
    }
)

# Unpack the results
total_loss = results[0]
attrition_loss = results[1]
attrition_accuracy = results[2]
department_loss = results[3]
department_accuracy = results[4]

# Print the results
print(f"\nTotal Loss: {total_loss:.4f}")
print(f"Attrition Loss: {attrition_loss:.4f} | Accuracy: {attrition_accuracy:.4f}")
print(f"Department Loss: {department_loss:.4f} | Accuracy: {department_accuracy:.4f}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 1.0000 - attrition_output_loss: 0.0070 - department_output_accuracy: 1.0000 - department_output_loss: 0.0063 - loss: 0.0134 

Total Loss: 0.0123
Attrition Loss: 0.0066 | Accuracy: 0.0054
Department Loss: 1.0000 | Accuracy: 1.0000


In [91]:
# Print the accuracy for both department and attrition
print(f"Attrition Accuracy:  {attrition_accuracy:.4f}")
print(f"Department Accuracy: {department_accuracy:.4f}")


Attrition Accuracy:  0.0054
Department Accuracy: 1.0000


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No, because 0.0054 accuracy for 'attrition' suggests the model is failing to learn anything meaningful about the "Yes" cases. 1.0000 accuracy for 'department' is also highly suspicious.
2. I used sigmoid and softmax.
3. Take steps to handle imbalance and add more layers.